In [272]:
# from openlocationcode import openlocationcode as olc
# olc.encode(47.365590, 8.524997)
#'8FVC9G8F+6X'

# @see datasets https://github.com/tensorflow/datasets
# @see https://stackoverflow.com/questions/44416764/loading-folders-of-images-in-tensorflow

import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pathlib

import time
default_timeit_steps = 1000


# data_dir = '/Volumes/dev/cda-5155-tensorflow/output_256px_grouped_grayscale/set_1'
data_dir = '/Volumes/dev/cda-5155-tensorflow/output/set_all'
validation_dir = '/Volumes/dev/cda-5155-tensorflow/output/set_validation'

data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))

print('Using tensorflow version: {}'.format(tf.__version__))
print('Using input dir: {} with {} images'.format(data_dir, image_count))


AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [252]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') ])
CLASS_NAMES


array(['76XVMM24+PV49G', '76XVMM25+J7M76', '76XVMM24+QHV37',
       '76XVJMX4+R4VJR', '76XVJMX3+877FW', '76XVJMX4+V4349',
       '76XVJMX2+CQFP4', '76XVJMX4+XPHX2', '76XVJMX5+W4424',
       '76XVJMX3+FJFV5', '76XVJMX2+9Q99G', '76XVJMX2+CPW45',
       '76XVJMX3+C8QQQ', '76XVJMX4+M47M9', '76XVJMX3+FP9FW',
       '76XVJMX3+CPQHJ', '76XVJMX3+9GHPW', '76XVJMX4+WC77W',
       '76XVMM24+QJCRV', '76XVMM25+84PJ2', '76XVJMX2+FF286',
       '76XVJMX4+WGP67', '76XVJMX4+R4VP8', '76XVJMX2+G4Q7X',
       '76XVMM24+4G5QJ', '76XVMM25+J3X49', '76XVMM24+7JQVM',
       '76XVMM24+6HQCH', '76XVJMX3+7JXHG', '76XVJMX3+C3P3M',
       '76XVMM24+MV9R5', '76XVJMX3+C77RF', '76XVMM24+3HRJ8',
       '76XVMM24+CJR4X', '76XVJMX4+W47W5', '76XVJMW3+JVRWJ',
       '76XVJMX4+XG5GP', '76XVJMX4+R4VWH', '76XVJMX4+M38V4',
       '76XVJMX4+WHR3R', '76XVJMX3+9QC65', '76XVJMX4+R4V8J',
       '76XVJMX3+9HXC2', '76XVJMX2+9CGRG', '76XVJMX4+XQ9HC',
       '76XVMM25+Q2P34', '76XVJMX4+R6JX4', '76XVJMX2+FR826',
       '76XVJMX2+CJCWQ',

In [253]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [254]:
image_generator

In [255]:
# Define some parameters for the loader:

epochs = 4

BATCH_SIZE = 128
IMG_HEIGHT = 192
IMG_WIDTH = 256
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
print('STEPS_PER_EPOCH: {}'.format(STEPS_PER_EPOCH))

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary'
                                                    )

val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                              batch_size=BATCH_SIZE,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              # classes = list(CLASS_NAMES)
                                                              class_mode='binary'
                                                             )

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

STEPS_PER_EPOCH: 5.0
Found 582 images belonging to 58 classes.
Found 13 images belonging to 3 classes.


In [256]:
# image_batch, label_batch = next(train_data_gen)
# show_batch(image_batch, label_batch)

In [257]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))


In [258]:
for f in list_ds.take(5):
  print(f.numpy())

b'/Volumes/dev/cda-5155-tensorflow/output/set_all/76XVJMX2+CHXXJ/thumb_0341_lat_29.648623_lon_-82.3485069.jpg'
b'/Volumes/dev/cda-5155-tensorflow/output/set_all/76XVJMX2+CQFP4/thumb_0416_lat_29.648565_lon_-82.3480739.jpg'
b'/Volumes/dev/cda-5155-tensorflow/output/set_all/76XVJMX3+CPQHJ/thumb_0566_lat_29.648588_lon_-82.3456319.jpg'
b'/Volumes/dev/cda-5155-tensorflow/output/set_all/76XVJMX2+CQFP4/thumb_0410_lat_29.648565_lon_-82.3480739.jpg'
b'/Volumes/dev/cda-5155-tensorflow/output/set_all/76XVJMX4+XG5GP/thumb_0003_lat_29.649888_lon_-82.343635.jpg'


In [259]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  # return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
  return img

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

print('AUTOTUNE: {}'.format(AUTOTUNE))
    

AUTOTUNE: -1


In [260]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)


In [261]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (192, 256, 3)
Label:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False False]


In [262]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


In [263]:
# speedup loading
# train_ds = prepare_for_training(labeled_ds)
# image_batch, label_batch = next(iter(train_ds))
# show_batch(image_batch.numpy(), label_batch.numpy())

In [264]:

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))


# timeit(train_data_gen)

In [265]:
# timeit(train_ds)

In [266]:

# tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter
# type(labeled_ds.)

# image_paths, labels = labeled_ds # load_base_data(...)
# epoch_size = len(image_paths)
# image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
# labels = tf.convert_to_tensor(labels)

# dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

# type(train_ds)

In [267]:
validation_ds, _ = next(train_data_gen)

len(validation_ds)


128

In [268]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 192, 256, 16)      448       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 96, 128, 16)       0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 96, 128, 32)       4640      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 48, 64, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 48, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 24, 32, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 49152)           

In [269]:
len(validation_ds)


128

In [270]:

total_train = 582 # len(train_ds)
total_val =  len(val_data_gen)

print('total_train: {}, total_val: {}'.format(total_train, total_val))

start = time.time()
print('started at: {}'.format(start))

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // BATCH_SIZE,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1 # total_val // BATCH_SIZE
)

total_train: 582, total_val: 1
started at: 1575350051.595467
Epoch 1/4
4/4 [==============================] - 37s 9s/step - loss: -271.9523 - accuracy: 0.0044 - val_loss: 1.1795 - val_accuracy: 0.3077
Epoch 2/4
4/4 [==============================] - 38s 10s/step - loss: -383.5456 - accuracy: 0.0044 - val_loss: 1.1795 - val_accuracy: 0.3077
Epoch 3/4
4/4 [==============================] - 37s 9s/step - loss: -370.5698 - accuracy: 0.0000e+00 - val_loss: 1.1795 - val_accuracy: 0.3077
Epoch 4/4
4/4 [==============================] - 45s 11s/step - loss: -373.8076 - accuracy: 0.0039 - val_loss: 1.1795 - val_accuracy: 0.3077


In [271]:
end = time.time()
duration = end-start
print("Done! Training duration {} s".format(duration))


Done! Training duration 157.51986408233643 s
